Влияние предобработки данных на модель xgboost, на примере датасета wine

In [410]:
# Библиотеки
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

import seaborn as sns

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import *
from sklearn.decomposition import PCA

import warnings
warnings.simplefilter('ignore')

In [411]:
# Импорт данных
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Series(wine.target)

In [412]:
# проверка пропущенных значений
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
dtypes: fl

In [413]:
# Проверка NA в целевой переменной
y.isna().sum()

0

In [414]:
# Количсетво уникальных значений каждого поля
df.nunique()

alcohol                         126
malic_acid                      133
ash                              79
alcalinity_of_ash                63
magnesium                        53
total_phenols                    97
flavanoids                      132
nonflavanoid_phenols             39
proanthocyanins                 101
color_intensity                 132
hue                              78
od280/od315_of_diluted_wines    122
proline                         121
dtype: int64

В датасете нет пропущенных значений 

Также нет категориальных полей(кроме целевой), т.к. в каждом поле большое количество различных числовых значений

In [415]:
# Датафрейм для результата
result = pd.DataFrame()

# Фунция для записи результата
def write_result(n_estimators, df, scale=True, scaler=None, pca=False, **kwargs):
    # Копия датафрейма
    step_df = df.copy()
    row_name = ''
    col_name = f'ac_{n_estimators}'
    
    if scale:
        # Если требуется, то масштабируем
        step_df = scaler(**kwargs).fit_transform(step_df)
        row_name += scaler.__name__ + str(kwargs)
        
    if pca:
        # Используем PCA c 2 компонентами, если нужно
        step_df = PCA(n_components=2).fit_transform(step_df)
        col_name += '_PCA'
        
    if df.isna().sum().sum() > 0:
        col_name += '_NA'
    
    scores = []
    # Прогоним 10 раз, т.к. результат очень сильно зависит от разделения выборки
    for _ in range(10):
        # Тренировочная и тестовая выборки
        X_train, X_test, y_train, y_test = train_test_split(step_df, y, test_size=0.25, 
                                                        stratify=y, shuffle=True)
        # Обучаем модель
        xgb = XGBClassifier(n_estimators=n_estimators, n_jobs=-1, num_class=3,
                    objective='multi:softmax', eval_metric='merror', seed=42)
        xgb.fit(X_train, y_train)

        predictions = xgb.predict(X_test)
        scores.append(accuracy_score(y_test, predictions))
    
    #записываем результат
    result.loc[row_name, col_name] = round(np.mean(scores), 4)


In [416]:
# Способы масштабирования
scalers_model = [StandardScaler, MinMaxScaler, MaxAbsScaler, 
                 QuantileTransformer, PowerTransformer]

In [417]:
# Проверим xgboost на разных типах масштабирования
def fit_scalers_models(**kwargs):
    write_result(**kwargs, scale=False)

    for scaler in scalers_model:
        if scaler == PowerTransformer:
            write_result(**kwargs, scaler=scaler, method='box-cox', standardize=False, )
            write_result(**kwargs, scaler=scaler, method='yeo-johnson', standardize=False)

        elif scaler == QuantileTransformer:
            write_result(**kwargs, scaler=scaler, output_distribution='uniform')
            write_result(**kwargs, scaler=scaler, output_distribution='normal')

        else:
            write_result(**kwargs, scaler=scaler, )
            

In [418]:
# Результаты по каждой модели масштабирования
for n in [5, 25]:
    fit_scalers_models(n_estimators=n, df=df)
result

,ac_5,ac_25
,0.9267,0.9689
StandardScaler{},0.9311,0.9533
MinMaxScaler{},0.9444,0.9689
MaxAbsScaler{},0.9556,0.9733
QuantileTransformer{'output_distribution': 'uniform'},0.9644,0.9756
QuantileTransformer{'output_distribution': 'normal'},0.9489,0.9533
"PowerTransformer{'method': 'box-cox', 'standardize': False}",0.9556,0.9756
"PowerTransformer{'method': 'yeo-johnson', 'standardize': False}",0.9489,0.9667


In [419]:
# Результаты с PCA
for n in [5, 25]:
    fit_scalers_models(n_estimators=n, df=df, pca=True)
result

,ac_5,ac_25,ac_5_PCA,ac_25_PCA
,0.9267,0.9689,0.7244,0.7311
StandardScaler{},0.9311,0.9533,0.9244,0.9422
MinMaxScaler{},0.9444,0.9689,0.9556,0.9511
MaxAbsScaler{},0.9556,0.9733,0.9333,0.9222
QuantileTransformer{'output_distribution': 'uniform'},0.9644,0.9756,0.9378,0.9667
QuantileTransformer{'output_distribution': 'normal'},0.9489,0.9533,0.9222,0.9578
"PowerTransformer{'method': 'box-cox', 'standardize': False}",0.9556,0.9756,0.9178,0.8889
"PowerTransformer{'method': 'yeo-johnson', 'standardize': False}",0.9489,0.9667,0.9222,0.9044


In [420]:
# Введем искусственно пропущенные значения, по 15 в каждый признак
df_with_na = df.copy()
for col in df_with_na.columns: 
    indexes = df_with_na.sample(15, random_state=5).index
    df_with_na.loc[indexes, col] = df_with_na.loc[indexes, col].apply(lambda x: np.nan)

df_with_na

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [421]:
# результаты с NA
for n in [5, 25]:
    fit_scalers_models(n_estimators=n, df=df_with_na)
result

,ac_5,ac_25,ac_5_PCA,ac_25_PCA,ac_5_NA,ac_25_NA
,0.9267,0.9689,0.7244,0.7311,0.8956,0.9200
StandardScaler{},0.9311,0.9533,0.9244,0.9422,0.8911,0.9044
MinMaxScaler{},0.9444,0.9689,0.9556,0.9511,0.8933,0.9067
MaxAbsScaler{},0.9556,0.9733,0.9333,0.9222,0.9067,0.9067
QuantileTransformer{'output_distribution': 'uniform'},0.9644,0.9756,0.9378,0.9667,0.9178,0.9356
QuantileTransformer{'output_distribution': 'normal'},0.9489,0.9533,0.9222,0.9578,0.8889,0.9156
"PowerTransformer{'method': 'box-cox', 'standardize': False}",0.9556,0.9756,0.9178,0.8889,0.9222,0.9089
"PowerTransformer{'method': 'yeo-johnson', 'standardize': False}",0.9489,0.9667,0.9222,0.9044,0.8933,0.9022


In [422]:
result.describe()

,ac_5,ac_25,ac_5_PCA,ac_25_PCA,ac_5_NA,ac_25_NA
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.946950,0.966950,0.904713,0.908050,0.901112,0.912513
std,0.012703,0.009026,0.073858,0.076369,0.012852,0.011029
min,0.926700,0.953300,0.724400,0.731100,0.888900,0.902200
25%,0.941075,0.963350,0.921100,0.900525,0.892750,0.906125
50%,0.948900,0.968900,0.923300,0.932200,0.894450,0.907800
75%,0.955600,0.973875,0.934425,0.952775,0.909475,0.916700
max,0.964400,0.975600,0.955600,0.966700,0.922200,0.935600


Полученные результаты: результаты очень сильно зависят от изначального разделения на тестовую и тренировочную, в таблице представлены средние значения accuracy за 10 прогонов. Как видно использование разных способов масштабирования влияет на результат, но не сильно, объясняется тем, что xgboost строился по древьям решений, которые не требуютмасштабирования данных. У нас изначально небольшое количество признаков, поэтому отбрасывать какую то информацию признаков (PCA) кажется не выгодным, метод только ухудшает результат. Т.к. в данных изначально не было пропущенных значений, а требовалось показать влияние предобработки на модель я искусственно добавил в каждый признак по 15 NA. Xgboost умеет обрабатывать NA, поэтому результат остается больше 90%, но конечно же результат хуже, чем у изначального набора.